In [35]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, ReLU, Flatten, Dense, Softmax
from tensorflow.keras.optimizers import Adam, SGD,RMSprop
import numpy as np
from sklearn.model_selection import train_test_split

In [36]:
tf.__version__

'2.4.0'

In [37]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.set_visible_devices(physical_devices[1], 'GPU')

# Data Pre-Processing

Open **kyu_train.csv** file and split the games into a list.
Every row of csv: `KL0000000001,B,B[pq],W[dd],B[dp],W[pd],B[jc],...`. 

Columns are:

    1. KL0000000001: Game ID
    2. B: Player's color
    3-... : Moves
    
We cropped only the moves to game list as:

In [38]:
df = open('./Training Dataset/kyu_train.csv').read().splitlines()
games = [i.split(',',2)[-1] for i in df]

Create a dictionary to convert the coordinates from characters to numbers

In [39]:
chars = 'abcdefghijklmnopqrs'
coordinates = {k:v for v,k in enumerate(chars)}
chartonumbers = {k:v for k,v in enumerate(chars)}
coordinates

{'a': 0,
 'b': 1,
 'c': 2,
 'd': 3,
 'e': 4,
 'f': 5,
 'g': 6,
 'h': 7,
 'i': 8,
 'j': 9,
 'k': 10,
 'l': 11,
 'm': 12,
 'n': 13,
 'o': 14,
 'p': 15,
 'q': 16,
 'r': 17,
 's': 18}

We decided to build a DCNN model in this tutorial. We create data samples by using every move in every game, meaning that the target is to predict the next move by feeding the previous state of the table in every game for every move. Therefore, we can collect much more data samples from games.

For the simplicity, we used 4 dimensional feature map to represent the data as below:
 1. Positions of black stones: mark them as 1 and the rest of the table as 0
 2. Positions of white stones: mark them as 1 and the rest of the table as 0
 3. Empty areas of the table: mark the empty areas as 1 and occupied areas as 0
 4. The last move in the table: mark the position of the last move as 1 and the rest as 0
 
Target value is a number between 0-361(19\*19). Later this will be one-hot encoded.

In [40]:
def prepare_input(moves):
    x = np.zeros((19,19,10))
    # x[:,:,6] = 1
    map = []
    for move in moves:
        color = move[0]
        column = coordinates[move[2]]
        row = coordinates[move[3]]
        map.append(row)
        map.append(column)
        if color == 'B':
            x[row,column,0] = 1
            x[row,column,2] = 1
            # x[:,:,6] = 0
        if color == 'W':
            x[row,column,1] = 1
            x[row,column,2] = 1
            # x[:,:,6] = 0
    if moves:
        last_move_column = coordinates[moves[-1][2]]
        last_move_row = coordinates[moves[-1][3]]
        x[row,column,3] = 1

        color = moves[-1][0]
        if color == 'B':
            x[last_move_row,last_move_column,4] = 1
        else:
            x[last_move_row,last_move_column,5] = 1

        # #最後一子的周邊分布
        # for dr, dc in [(last_move_row, last_move_column+1), (last_move_row, last_move_column-1), (last_move_row+1, last_move_column), (last_move_row-1, last_move_column)]:   
        #     if 0 <= dr < 19 and 0<= dc < 19 and not x[dr,dc,0]:
        #         x[dr,dc,9]=1
        # #氣
        # for dr, dc in [(last_move_row, last_move_column+1), (last_move_row, last_move_column-1), (last_move_row+1, last_move_column), (last_move_row-1, last_move_column)]:
        #     if color == 'B':
        #         if 0 <= dr < 19 and 0<= dc < 19 and not x[dr,dc,5]:
        #             x[dr,dc,7]=1 
        #     else:
        #         if 0 <= dr < 19 and 0<= dc < 19 and not x[dr,dc,4]:
        #             x[dr,dc,8]=1

        
    for i in range(0, len(map), 2):
        qi = 4
        for dr, dc in [(map[i], map[i+1]+1), (map[i], map[i+1]-1), (map[i]+1, map[i+1]), (map[i]-1, map[i+1])]:   
            if 0 <= dr < 19 and 0<= dc < 19 and not x[dr,dc,0]:
                qi-=1
        if qi==1:
            x[map[i],map[i+1],6]=1
        if qi==2:
            x[map[i],map[i+1],7]=1
        if qi==3:
            x[map[i],map[i+1],8]=1
        if qi==4:
            x[map[i],map[i+1],9]=1

    x[:,:,2] = np.where(x[:,:,2] == 0, 1, 0)
    return x

def prepare_label(move):
    column = coordinates[move[2]]
    row = coordinates[move[3]]
    return column*19+row

In [41]:
# Check how many samples can be obtained
n_games = 0
n_moves = 0
for game in games:
    n_games += 1
    moves_list = game.split(',')
    for move in moves_list:
        n_moves += 1
print(f"Total Games: {n_games}, Total Moves: {n_moves}")

Total Games: 118500, Total Moves: 27135638


The code below is run for baseline model only by using only the first 500 games from the dataset. You might need to create a data generator to use complete dataset. Otherwise your RAM might not enough to store all (If you run the code on free version of Google Colab, it will crash above 500 game samples).

In [42]:
# x = []
# y = []
# for game in games[:7000]:
#     moves_list = game.split(',')
#     for count, move in enumerate(moves_list):
#         x.append(prepare_input(moves_list[:count]))
#         y.append(prepare_label(moves_list[count]))
# x = np.array(x)
# y = np.array(y)

In [43]:
# x.shape

In [44]:
# y.shape

In [45]:
# y_one_hot = tf.one_hot(y, depth=19*19)

Dataset splitting: 90% Training, 10% validation

In [46]:
# x_train, x_val, y_train, y_val = train_test_split(x, y_one_hot.numpy(), test_size=0.10)

# Training

### Simple DCNN Model:

In [58]:
# def create_model():
#     inputs = Input(shape=(19, 19, 4))
#     outputs = Conv2D(kernel_size=7, filters=32, padding='same', activation='relu')(inputs)
#     outputs = Conv2D(kernel_size=7, filters=32, padding='same', activation='relu')(outputs)
#     outputs = Conv2D(kernel_size=5, filters=32, padding='same', activation='relu')(outputs)
#     outputs = Conv2D(kernel_size=5, filters=32, padding='same', activation='relu')(outputs)
#     outputs = Conv2D(kernel_size=3, filters=32, padding='same', activation='relu')(outputs)
#     outputs = Conv2D(kernel_size=3, filters=1, padding='same', activation='relu')(outputs)
#     outputs = Flatten()(outputs)
#     outputs = Softmax()(outputs)
#     model = Model(inputs, outputs)
    
#     opt = Adam(learning_rate=0.001)
#     model.compile(optimizer=opt,
#                   loss='categorical_crossentropy',
#                   metrics=['accuracy'])
#     return model

from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, Add, Input, Flatten, Dense, Softmax

def residual_block(inputs, filters, kernel_size=3, strides=1):
    shortcut = inputs

    outputs = Conv2D(filters, kernel_size=kernel_size, strides=strides, padding='same')(inputs)
    outputs = BatchNormalization()(outputs)
    outputs = Activation('relu')(outputs)

    outputs = Conv2D(filters, kernel_size=kernel_size, strides=strides, padding='same')(outputs)
    outputs = BatchNormalization()(outputs)

    if shortcut.shape[-1] != filters:
        shortcut = Conv2D(filters, kernel_size=1, strides=strides, padding='same')(shortcut)
    
    outputs = Add()([outputs, shortcut])
    outputs = Activation('relu')(outputs)
    return outputs

def create_model():
    inputs = Input(shape=(19, 19, 10))

    outputs = Conv2D(kernel_size=7, filters=32, padding='same', activation='relu')(inputs)
    outputs = Conv2D(kernel_size=7, filters=32, padding='same', activation='relu')(outputs)
    outputs = Conv2D(kernel_size=5, filters=32, padding='same', activation='relu')(outputs)
    outputs = Conv2D(kernel_size=5, filters=32, padding='same', activation='relu')(outputs)
    outputs = Conv2D(kernel_size=3, filters=32, padding='same', activation='relu')(outputs)
    outputs = Conv2D(kernel_size=3, filters=32, padding='same', activation='relu')(outputs)
    outputs = Conv2D(kernel_size=3, filters=32, padding='same', activation='relu')(outputs)

    # Adding a residual block
    outputs = residual_block(outputs, filters=32)
    outputs = residual_block(outputs, filters=32)
    outputs = residual_block(outputs, filters=32)


    outputs = Conv2D(kernel_size=3, filters=1, padding='same')(outputs)
    outputs = Flatten()(outputs)
    outputs = Softmax()(outputs)
    
    model = Model(inputs, outputs)
    
    # opt = tf.keras.optimizers.Adam(learning_rate=0.001)
    opt = opt = RMSprop(learning_rate=0.001, rho=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [59]:
model = create_model()
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           [(None, 19, 19, 10)] 0                                            
__________________________________________________________________________________________________
conv2d_119 (Conv2D)             (None, 19, 19, 32)   15712       input_10[0][0]                   
__________________________________________________________________________________________________
conv2d_120 (Conv2D)             (None, 19, 19, 32)   50208       conv2d_119[0][0]                 
__________________________________________________________________________________________________
conv2d_121 (Conv2D)             (None, 19, 19, 32)   25632       conv2d_120[0][0]                 
____________________________________________________________________________________________

In [ ]:
def games_generator():
    for game in games:
        yield from process_game(game)

def process_game(game):
    moves_list = game.split(',')
    for count, move in enumerate(moves_list):
        input_data = prepare_input(moves_list[:count])
        label = prepare_label(moves_list[count])
        yield input_data, label


games_dataset = tf.data.Dataset.from_generator(
    games_generator,
    output_types=(tf.float32, tf.int32),
    output_shapes=([19, 19, 10], [])
)


# 將數據集分成訓練和驗證集
val_dataset = games_dataset.take(11000) 
train_dataset = games_dataset.skip(11000)

# 對數據集進行批處理和預處理
train_dataset = train_dataset.batch(512).map(lambda x, y: (x, tf.one_hot(y, depth=19*19)))
val_dataset = val_dataset.batch(512).map(lambda x, y: (x, tf.one_hot(y, depth=19*19)))


from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint_path = "test_model_bestkyu.h5"
checkpoint = ModelCheckpoint(checkpoint_path, 
                             monitor='val_loss',  # 監控準確度
                             save_best_only=True,     # 只保存最佳模型
                             mode='min',              # 目標是最大化監控指標
                             verbose=1)

# 訓練模型
history = model.fit(
    train_dataset,
    epochs=25,
    validation_data=val_dataset,
    steps_per_epoch=1000,
    callbacks=[checkpoint]
)

Epoch 1/25


2023-11-25 17:28:18.101677: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.8


: 

In [ ]:
history = model.fit(
    x = x_train, 
    y = y_train,
    batch_size = 512,
    epochs = 25,
    validation_data=(x_val, y_val),
)

2023-11-21 11:05:52.875000: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 12450271968 exceeds 10% of free system memory.
2023-11-21 11:05:56.274156: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2023-11-21 11:05:56.276709: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2112000000 Hz


Epoch 1/25


2023-11-21 11:05:56.516477: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.8
2023-11-21 11:05:59.535158: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2023-11-21 11:05:59.536415: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11


2807/2807 [==============================] - 78s 26ms/step - loss: 4.2914 - accuracy: 0.1198 - val_loss: 3.2879 - val_accuracy: 0.3041
Epoch 2/25
2807/2807 [==============================] - 73s 26ms/step - loss: 3.1513 - accuracy: 0.3232 - val_loss: 2.9832 - val_accuracy: 0.3496
Epoch 3/25
2807/2807 [==============================] - 73s 26ms/step - loss: 2.9346 - accuracy: 0.3541 - val_loss: 2.9055 - val_accuracy: 0.3601
Epoch 4/25
2807/2807 [==============================] - 73s 26ms/step - loss: 2.8232 - accuracy: 0.3692 - val_loss: 2.8380 - val_accuracy: 0.3691
Epoch 5/25
2807/2807 [==============================] - 73s 26ms/step - loss: 2.7509 - accuracy: 0.3781 - val_loss: 2.7303 - val_accuracy: 0.3844
Epoch 6/25
2807/2807 [==============================] - 73s 26ms/step - loss: 2.6928 - accuracy: 0.3866 - val_loss: 2.6953 - val_accuracy: 0.3902
Epoch 7/25
2807/2807 [==============================] - 73s 26ms/step - loss: 2.6516 - accuracy: 0.3922 - val_loss: 2.6859 - val_accura

In [16]:
model.save('./model_kyu_tutorial5.h5')

## ALL DONE!

For using the model and creating a submission file, follow the notebook **Create Public Upload CSV.ipynb**

# End of Tutorial

You are free to use more modern NN architectures, a better pre-processing, feature extraction methods to achieve much better accuracy!